In [1]:
import pandas as pd
import csv
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import Session, declarative_base

In [2]:
engine = create_engine('sqlite:///dictionary.sqlite')
Base = declarative_base()

In [3]:
class Word(Base):
    __tablename__ = 'word'
    
    id = Column(Integer, primary_key=True)
    word = Column(String(45))

class Definition(Base):
    __tablename__ = 'definition'
    
    id = Column(Integer, primary_key=True)
    word_id = Column(Integer, ForeignKey('word.id'), nullable=False)
    type = Column(String(10))
    definition = Column(String)

In [ ]:
with Session(engine) as s:
    definitions = s.query(Definition).all()
types = []
for definition in definitions:
    if definition.type not in types:
        types.append(definition.type)

print(types)

In [ ]:
 type_dict = {}
flag = []
for t in types:
    type_dict[t] = []
    if 'adv.' in t:
        type_dict[t].append('adverb')
    elif 'v.' in t:
        type_dict[t].append('verb')
    if 'pron' in t:
        type_dict[t].append('pronoun')
    elif 'n.' in t or 'n .' == t or 'N.' == t:
        type_dict[t].append('noun')
    if 'interj' in t:
        type_dict[t].append('interjection')
    if 'a.' in t:
        type_dict[t].append('adjective')
    if 'con' in t:
        type_dict[t].append('conjunction')
    if 'prep.' in t:
        type_dict[t].append('preposition')
    if 'comp' in t.lower():
        type_dict[t].append('comparison')
    if 'object' in t:
        type_dict[t].append('noun')
    if 'p. p.' in t and ('verb' in type_dict[t]) == False:
        type_dict[t].append('verb')
    if 'p. pr.' in t and ('verb' in type_dict[t]) == False:
        type_dict[t].append('verb')
    if 'imp.' in t and ('verb' in type_dict[t]) == False:
        type_dict[t].append('verb')
    if 'obj.' == t:
        type_dict[t].append('noun')
    if '' == t or '/' == t or '/.' == t:
        type_dict[t].append('none')
    if len(type_dict[t]) == 0:
        type_dict[t].append(t)
        type_dict[t].append('other')

In [ ]:
type_dict

In [4]:
class Typemap(Base):
    __tablename__ = 'typemap'

    id = Column(Integer, primary_key=True)
    base = Column(String(10))
    converted = Column(String(10))

In [5]:
Base.metadata.create_all(engine)

In [ ]:
for bt, cts in type_dict.items():
    for ct in cts:
        with Session(engine) as s:
            inst = Typemap(base=bt, converted=ct)
            s.add(inst)
            s.commit()

In [ ]:
with Session(engine) as s:
    q = s.query(Typemap).all()

for t in q:
    print(f'{t.base} ---> {t.converted}')

In [ ]:
len(q)

In [9]:
with Session(engine) as s:
    s.query(Definition).join(Typemap, Definition.type == Typemap.base).filter(Typemap.converted == 'comparison').all()